In [1]:
import pandas as pd
import cv2
import matplotlib
import mediapipe as mp
from matplotlib import pyplot as plt
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import sys
import os
import shutil
import warnings
from torch.optim import Adam
import torchmetrics
import time
from sklearn.metrics import accuracy_score

C:\Users\dfoda\anaconda3\envs\ASLRecognition\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Не найдена указанная процедура'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

In [3]:
IMG_WIDTH = IMG_HEIGHT = SIZE = 100
BATCH_SIZE = 64

In [4]:
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor()
])

In [6]:
dataset = ImageFolder('data/processed_images_hands_init/', transform=transform)

In [7]:
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size

train_dataset, valid_dataset = random_split(dataset, (train_size, valid_size))

In [8]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
def prepare_model(model, num_classes=29, lr=0.001, device=None):

    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc.parameters():
        param.requires_grad = True

    optimizer = Adam(model.fc.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()

    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    accuracy = torchmetrics.Accuracy(task='MULTICLASS', num_classes=29)
    accuracy.to(device)
    
    return model, criterion, optimizer, accuracy, device

In [10]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'models/vgg16_best_model.pth')
        self.val_loss_min = val_loss


In [11]:
def train_model(model, num_epochs, train_loader, val_loader, optimizer, loss_fun, accuracy, device, early_stopping=False):
    model.to(device)
    print(model.__class__.__name__)
    
    if early_stopping:
        early_stopping = EarlyStopping(patience=40, verbose=True)
        
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        num_batches = 0
        
        accuracy.reset()  # Reset metric at the beginning of each epoch
        
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", total=len(train_loader)):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fun(outputs, labels)
            loss.backward()
            optimizer.step()
            
            accuracy.update(outputs, labels)
            running_loss += loss.item()
            num_batches += 1
        
        average_loss = running_loss / num_batches
        epoch_accuracy = accuracy.compute()
        
        model.eval()
        val_loss = 0.0
        val_batches = 0
        val_accuracy = 0.0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}", total=len(val_loader)):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = loss_fun(outputs, labels)
                val_loss += loss.item()
                val_batches += 1
                val_accuracy += torch.sum(torch.argmax(outputs, dim=1) == labels).item()
        
        average_val_loss = val_loss / val_batches
        val_accuracy = val_accuracy / len(val_loader.dataset)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Val Loss: {average_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

        if early_stopping:
            early_stopping(average_val_loss, model)
            if early_stopping.early_stop:
                print("Early stopping")
                break


In [40]:
resnet50 = models.resnet50(pretrained=True)

C:\Users\dfoda\anaconda3\envs\ASLRecognition\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dfoda\anaconda3\envs\ASLRecognition\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [41]:
model, criterion, optimizer, accuracy, device = prepare_model(resnet50)

In [51]:
num_epochs = 20

In [52]:
resnet50_trained = train_model(model, num_epochs, train_loader, valid_loader, optimizer, criterion, accuracy, device)

Validation Epoch 1/20: 100%|████████████████████████████████████████████████████████▋| 225/226 [01:34<00:00,  2.47it/s]C:\Users\dfoda\anaconda3\envs\ASLRecognition\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Validation Epoch 1/20: 100%|█████████████████████████████████████████████████████████| 226/226 [01:35<00:00,  2.38it/s]


Epoch [1/20], Loss: 0.5252, Accuracy: 0.8715, Val Loss: 0.2479, Val Accuracy: 0.9299


Validation Epoch 2/20: 100%|█████████████████████████████████████████████████████████| 226/226 [01:33<00:00,  2.42it/s]


Epoch [2/20], Loss: 0.2072, Accuracy: 0.9435, Val Loss: 0.1836, Val Accuracy: 0.9435


Validation Epoch 3/20: 100%|█████████████████████████████████████████████████████████| 226/226 [01:32<00:00,  2.46it/s]


...

Epoch [18/20], Loss: 0.0430, Accuracy: 0.9875, Val Loss: 0.0778, Val Accuracy: 0.9752


Validation Epoch 19/20: 100%|████████████████████████████████████████████████████████| 226/226 [01:32<00:00,  2.44it/s]


Epoch [19/20], Loss: 0.0410, Accuracy: 0.9884, Val Loss: 0.0629, Val Accuracy: 0.9799


Validation Epoch 20/20: 100%|████████████████████████████████████████████████████████| 226/226 [01:31<00:00,  2.46it/s]

Epoch [20/20], Loss: 0.0385, Accuracy: 0.9892, Val Loss: 0.0626, Val Accuracy: 0.9806


In [92]:
torch.save(model.state_dict(), 'models/MobileNet_20.pth')

In [54]:
def prepare_vgg16(model, num_classes=29, lr=0.001, device=None):
    num_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_features, num_classes)

    for param in model.parameters():
        param.requires_grad = False

    for param in model.classifier[-1].parameters():
        param.requires_grad = True

    optimizer = Adam(model.classifier[-1].parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    accuracy = torchmetrics.Accuracy(task='MULTICLASS', num_classes=29)
    accuracy.to(device)
    
    return model, criterion, optimizer, accuracy, device

In [27]:
vgg16 = models.vgg16(pretrained=True)
model, criterion, optimizer, accuracy, device = prepare_vgg16(vgg16)

In [82]:
train_model(model, num_epochs, train_loader, valid_loader, optimizer, criterion, accuracy, device, True)

VGG


Validation Epoch 1/20: 100%|█████████████████████████████████████████████████████████| 226/226 [02:13<00:00,  1.69it/s]


Epoch [1/20], Loss: 0.4886, Accuracy: 0.8539, Val Loss: 0.1681, Val Accuracy: 0.9591
Validation loss decreased (inf --> 0.168092).  Saving model ...


Validation Epoch 2/20: 100%|█████████████████████████████████████████████████████████| 226/226 [02:12<00:00,  1.71it/s]


Epoch [2/20], Loss: 0.2668, Accuracy: 0.9125, Val Loss: 0.1150, Val Accuracy: 0.9719
Validation loss decreased (0.168092 --> 0.114986).  Saving model ...


...

Epoch [18/20], Loss: 0.1927, Accuracy: 0.9412, Val Loss: 0.0396, Val Accuracy: 0.9890
Validation loss decreased (0.045271 --> 0.039566).  Saving model ...


Validation Epoch 19/20: 100%|████████████████████████████████████████████████████████| 226/226 [02:16<00:00,  1.66it/s]


Epoch [19/20], Loss: 0.1899, Accuracy: 0.9412, Val Loss: 0.0501, Val Accuracy: 0.9847
EarlyStopping counter: 1 out of 40


Validation Epoch 20/20: 100%|████████████████████████████████████████████████████████| 226/226 [02:14<00:00,  1.69it/s]

Epoch [20/20], Loss: 0.1895, Accuracy: 0.9415, Val Loss: 0.0490, Val Accuracy: 0.9877
EarlyStopping counter: 2 out of 40


In [55]:
def prepare_mobilenetv2(model, num_classes=29, lr=0.001, device=None):
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)

    for param in model.features.parameters():
        param.requires_grad = False

    optimizer = Adam(model.classifier.parameters(), lr=lr)

    criterion = nn.CrossEntropyLoss()
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    accuracy = torchmetrics.Accuracy(task='MULTICLASS', num_classes=29)
    accuracy.to(device)
    
    return model, criterion, optimizer, accuracy, device

In [44]:
mobilenet = models.mobilenet_v2(pretrained=True)
model, criterion, optimizer, accuracy, device = prepare_mobilenetv2(mobilenet)

In [91]:
train_model(model, num_epochs, train_loader, valid_loader, optimizer, criterion, accuracy, device, True)

MobileNetV2


Validation Epoch 1/20: 100%|█████████████████████████████████████████████████████████| 226/226 [01:28<00:00,  2.57it/s]


Epoch [1/20], Loss: 0.4978, Accuracy: 0.8787, Val Loss: 0.1808, Val Accuracy: 0.9587
Validation loss decreased (inf --> 0.180827).  Saving model ...


Validation Epoch 2/20: 100%|█████████████████████████████████████████████████████████| 226/226 [01:30<00:00,  2.48it/s]


Epoch [2/20], Loss: 0.1989, Accuracy: 0.9442, Val Loss: 0.1392, Val Accuracy: 0.9633
Validation loss decreased (0.180827 --> 0.139204).  Saving model ...


...

Epoch [18/20], Loss: 0.1041, Accuracy: 0.9657, Val Loss: 0.0710, Val Accuracy: 0.9800
EarlyStopping counter: 1 out of 40


Validation Epoch 19/20: 100%|████████████████████████████████████████████████████████| 226/226 [01:23<00:00,  2.72it/s]


Epoch [19/20], Loss: 0.1048, Accuracy: 0.9651, Val Loss: 0.0757, Val Accuracy: 0.9807
EarlyStopping counter: 2 out of 40


Validation Epoch 20/20: 100%|████████████████████████████████████████████████████████| 226/226 [01:20<00:00,  2.80it/s]

Epoch [20/20], Loss: 0.1026, Accuracy: 0.9669, Val Loss: 0.0619, Val Accuracy: 0.9830
Validation loss decreased (0.062538 --> 0.061878).  Saving model ...


In [12]:
from PIL import Image
import numpy as np

def predict_image_class(model, image, transform, class_names):
    model.eval()
    image = transform(image).unsqueeze(0)
    image = image.to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]


In [13]:
def detect_hand(image, hands, padding=20, target_size=(300, 300), bg_color=(128, 128, 128)):
    rgb_image = image
    rgb_image.flags.writeable = False

    results = hands.process(rgb_image)

    rgb_image.flags.writeable = True

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            h, w, _ = image.shape
            x_min, y_min = w, h
            x_max, y_max = 0, 0

            landmarks = []

            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * w), int(landmark.y * h)
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)
                landmarks.append((x, y, landmark.z))

            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)

            hand_image = image[y_min:y_max, x_min:x_max]

            hand_h, hand_w, _ = hand_image.shape
            scale = min(target_size[0] / hand_w, target_size[1] / hand_h)
            new_w = int(hand_w * scale)
            new_h = int(hand_h * scale)
            resized_hand_image = cv2.resize(hand_image, (new_w, new_h))

            result_image = np.full((target_size[1], target_size[0], 3), bg_color, dtype=np.uint8)
            x_offset = (target_size[0] - new_w) // 2
            y_offset = (target_size[1] - new_h) // 2
            result_image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_hand_image

            return result_image, results, (x_min, y_min, scale, x_offset, y_offset), landmarks
    return None, None, None, None


In [14]:
cap = cv2.VideoCapture(0)
hands = mp.solutions.hands.Hands(max_num_hands=1)
draw = mp.solutions.drawing_utils

class_names = dataset.classes

In [23]:
def start_camera(model, hands):
    total_images = 0
    total_time = 0
    while True:
        success, image = cap.read()
        image = cv2.flip(image, 1)
        imageRGB = image

        hand_pattern, results, bbox_params, original_landmarks = detect_hand(image, hands)
        if results:
            for handLms in results.multi_hand_landmarks:
                draw.draw_landmarks(image, handLms, mp.solutions.hands.HAND_CONNECTIONS)

                x = int(handLms.landmark[0].x * image.shape[1])
                y = int(handLms.landmark[0].y * image.shape[0])
        
        if hand_pattern is not None:
            imageRGB = cv2.resize(hand_pattern, (150, 150))
        else:
            imageRGB = cv2.resize(imageRGB, (150, 150))
    
        if results:
            if results.multi_hand_landmarks:
                for handLms in results.multi_hand_landmarks:
                    draw.draw_landmarks(image, handLms, mp.solutions.hands.HAND_CONNECTIONS)
                    pil_image = Image.fromarray(imageRGB)
                    time_s = time.time()
                    predicted_class = predict_image_class(model, pil_image, transform, class_names)
                    total_time += time.time() - time_s
                    total_images += 1
                    cv2.putText(image, predicted_class, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (224, 255, 255), 2, cv2.LINE_AA)
                    dynamic_console_output(f"Predicted class: {predicted_class}")

        image_out = image

        height, width, _ = imageRGB.shape
        height_out, width_out, _ = image_out.shape
    
        x_offset = width_out - width
        y_offset = 0
        
        image_out[y_offset:y_offset+height, x_offset:x_offset+width] = imageRGB

        cv2.imshow('Hand', image_out)
        
        if cv2.waitKey(33) != -1:
            cv2.destroyAllWindows()
            # break
            return total_time / total_images

In [16]:
def dynamic_console_output(log):
    terminal_width = shutil.get_terminal_size().columns
    log = log.ljust(terminal_width)
    sys.stdout.write("\r\033[K" + log)
    sys.stdout.flush()

In [51]:
device = 'cpu'

In [46]:
resnet50_t = resnet50
resnet50_t.load_state_dict(torch.load('models/resnet50_20.pth'))

<All keys matched successfully>

In [21]:
vgg16_t = vgg16
vgg16_t.load_state_dict(torch.load('models/vgg16_20.pth'))

<All keys matched successfully>

In [45]:
mobilenet_t = mobilenet
mobilenet_t.load_state_dict(torch.load('models/mobilenet_20.pth'))

<All keys matched successfully>

In [132]:
start_camera(resnet50_t, hands)

Predicted class: S                                                                                                      

0.027346482331102543

In [38]:
start_camera(vgg16_t, hands)

Predicted class: M                                                                                                      

0.06815091768900554

In [134]:
start_camera(mobilenet_t, hands)

Predicted class: E                                                                                                      

0.013859311944430637

In [53]:
cap.release()

In [48]:
def predict_images_in_folder(folder_path, model):
    predictions = []
    true_labels = []
    
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    
    for subfolder in subfolders:
        class_name = os.path.basename(subfolder)
        print(f'Processing images in class: {class_name}')
     
        files = [f for f in os.listdir(subfolder) if f.endswith('.jpg') or f.endswith('.png')]
        
        for file in files:
            image_path = os.path.join(subfolder, file)
            image = cv2.imread(image_path)

            hand_pattern, results_pattern, bbox_params, original_landmarks = detect_hand(image, hands)
            if hand_pattern is not None:
                imageRGB = cv2.resize(hand_pattern, (SIZE, SIZE))
            else:
                imageRGB = cv2.resize(image, (SIZE, SIZE))
            if results_pattern:
                if results_pattern.multi_hand_landmarks:
                    pil_image = Image.fromarray(imageRGB)
                    predicted_class = predict_image_class(model, pil_image, transform, class_names)
                else:
                    predicted_class = 'nothing'
            else:
                predicted_class = 'nothing'
            predictions.append(predicted_class)
            true_labels.append(class_name)
            
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Accuracy: {accuracy}')

In [52]:
from sklearn.metrics import accuracy_score
path = 'data/test_images'
predict_images_in_folder(path, mobilenet_t)

Processing images in class: A
Processing images in class: B
Processing images in class: C
Processing images in class: D
Processing images in class: del
Processing images in class: E
Processing images in class: F
Processing images in class: G
Processing images in class: H
Processing images in class: I
Processing images in class: J
Processing images in class: K
Processing images in class: L
Processing images in class: M
Processing images in class: N
Processing images in class: nothing
Processing images in class: O
Processing images in class: P
Processing images in class: Q
Processing images in class: R
Processing images in class: S
Processing images in class: space
Processing images in class: T
Processing images in class: U
Processing images in class: V
Processing images in class: W
Processing images in class: X
Processing images in class: Y
Processing images in class: Z
Accuracy: 0.2824137931034483
